# ANOVA Analysis: Diet Dataset

## Research Question
**Is one diet statistically more effective than the others, or is the weight loss variation just random?**

### Dataset Overview
- **Source**: University of Sheffield Dataset Archive
- **Description**: Weight loss measurements for people following one of three different diets
- **Groups**: Diet 1, Diet 2, Diet 3
- **Dependent Variable**: Weight loss (in kg or lbs)
- **Independent Variable**: Diet type (categorical)

### Dataset URL
We'll fetch this from the Sheffield University dataset repository or create a sample dataset based on typical diet study results.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway, shapiro, levene, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Load and Prepare the Data

We'll try to download from the Sheffield repository, or create realistic sample data based on typical diet study results.

In [ ]:
# Try to load from URL, otherwise create sample data
try:
    # Sheffield dataset URL (adjust if different)
    url = "https://www.sheffield.ac.uk/polopoly_fs/1.570199!/file/stcp-Rdataset-Diet.csv"
    df = pd.read_csv(url)
    print("✓ Dataset loaded from Sheffield University")
except:
    print("Creating representative sample data based on diet study patterns...\n")
    
    # Create realistic diet data based on research patterns
    np.random.seed(42)
    
    # Diet 1: Low-carb (typically 8-10 lbs weight loss)
    diet1_n = 24
    diet1 = np.random.normal(9, 2.5, diet1_n)
    
    # Diet 2: Low-fat (typically 6-8 lbs weight loss)
    diet2_n = 27
    diet2 = np.random.normal(7, 2.2, diet2_n)
    
    # Diet 3: Mediterranean (typically 10-12 lbs weight loss)
    diet3_n = 22
    diet3 = np.random.normal(11, 2.8, diet3_n)
    
    # Ensure no negative values
    diet1 = np.maximum(diet1, 0.5)
    diet2 = np.maximum(diet2, 0.5)
    diet3 = np.maximum(diet3, 0.5)
    
    # Create DataFrame
    df = pd.DataFrame({
        'weight_loss': np.concatenate([diet1, diet2, diet3]),
        'diet': ['Diet 1']*diet1_n + ['Diet 2']*diet2_n + ['Diet 3']*diet3_n,
        'person_id': range(1, diet1_n + diet2_n + diet3_n + 1)
    })
    
    print("✓ Sample dataset created")

print(f"\nDataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print("\nFirst 10 rows:")
df.head(10)

In [ ]:
# Dataset information
print("Dataset Information:")
print("="*60)
df.info()

print("\n" + "="*60)
print("Missing Values:")
print(df.isnull().sum())

print("\n" + "="*60)
print("Diet Group Distribution:")
print(df['diet'].value_counts().sort_index())

# Check for any negative or unrealistic values
print("\n" + "="*60)
print("Data Quality Check:")
print(f"Minimum weight loss: {df['weight_loss'].min():.2f}")
print(f"Maximum weight loss: {df['weight_loss'].max():.2f}")
print(f"Any negative values: {(df['weight_loss'] < 0).any()}")

## 2. Exploratory Data Analysis

In [ ]:
# Overall statistics
print("Overall Weight Loss Statistics:")
print("="*60)
print(df['weight_loss'].describe())

print("\n" + "="*60)
print("Statistics by Diet Group:")
print("="*60)
group_stats = df.groupby('diet')['weight_loss'].describe()
print(group_stats)

In [ ]:
# Detailed group statistics
print("\nDetailed Statistics by Diet:")
print("="*60)

summary_stats = pd.DataFrame({
    'Count': df.groupby('diet')['weight_loss'].count(),
    'Mean': df.groupby('diet')['weight_loss'].mean(),
    'Median': df.groupby('diet')['weight_loss'].median(),
    'Std Dev': df.groupby('diet')['weight_loss'].std(),
    'Variance': df.groupby('diet')['weight_loss'].var(),
    'Min': df.groupby('diet')['weight_loss'].min(),
    'Max': df.groupby('diet')['weight_loss'].max(),
    'Range': df.groupby('diet')['weight_loss'].apply(lambda x: x.max() - x.min()),
    'SE': df.groupby('diet')['weight_loss'].apply(lambda x: x.std() / np.sqrt(len(x)))
})

print(summary_stats.round(3))

# Calculate percentage of total participants in each diet
print("\n" + "="*60)
print("Group Size Percentages:")
print("="*60)
pct = (df.groupby('diet').size() / len(df) * 100).round(1)
for diet, percent in pct.items():
    print(f"{diet}: {percent}%")

## 3. Data Visualization

In [ ]:
# Comprehensive visualization
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Color palette
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

# 1. Box Plot
ax1 = fig.add_subplot(gs[0, 0])
df.boxplot(column='weight_loss', by='diet', ax=ax1, patch_artist=True)
ax1.set_title('Box Plot: Weight Loss by Diet', fontweight='bold')
ax1.set_xlabel('Diet Type')
ax1.set_ylabel('Weight Loss (lbs)')
plt.sca(ax1)
plt.xticks(rotation=0)

# 2. Violin Plot
ax2 = fig.add_subplot(gs[0, 1])
parts = ax2.violinplot([df[df['diet']==d]['weight_loss'].values for d in sorted(df['diet'].unique())],
                       positions=range(len(df['diet'].unique())),
                       showmeans=True, showmedians=True)
ax2.set_xticks(range(len(df['diet'].unique())))
ax2.set_xticklabels(sorted(df['diet'].unique()))
ax2.set_title('Violin Plot: Weight Loss Distribution', fontweight='bold')
ax2.set_xlabel('Diet Type')
ax2.set_ylabel('Weight Loss (lbs)')
ax2.grid(axis='y', alpha=0.3)

# 3. Swarm Plot with Box overlay
ax3 = fig.add_subplot(gs[0, 2])
sns.boxplot(data=df, x='diet', y='weight_loss', ax=ax3, palette=colors, width=0.5)
sns.swarmplot(data=df, x='diet', y='weight_loss', ax=ax3, color='black', alpha=0.5, size=4)
ax3.set_title('Individual Data Points with Box Plot', fontweight='bold')
ax3.set_xlabel('Diet Type')
ax3.set_ylabel('Weight Loss (lbs)')

# 4. Bar Plot with Error Bars (Mean ± SE)
ax4 = fig.add_subplot(gs[1, 0])
means = df.groupby('diet')['weight_loss'].mean()
sems = df.groupby('diet')['weight_loss'].sem()
x_pos = range(len(means))
bars = ax4.bar(x_pos, means, yerr=sems, capsize=10, alpha=0.8, color=colors, edgecolor='black', linewidth=1.5)
ax4.set_xticks(x_pos)
ax4.set_xticklabels(means.index)
ax4.set_title('Mean Weight Loss ± SE by Diet', fontweight='bold')
ax4.set_xlabel('Diet Type')
ax4.set_ylabel('Mean Weight Loss (lbs)')
ax4.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, (m, s) in enumerate(zip(means, sems)):
    ax4.text(i, m + s + 0.3, f'{m:.1f}', ha='center', va='bottom', fontweight='bold')

# 5. Histogram by Diet
ax5 = fig.add_subplot(gs[1, 1])
for i, diet in enumerate(sorted(df['diet'].unique())):
    diet_data = df[df['diet'] == diet]['weight_loss']
    ax5.hist(diet_data, alpha=0.6, label=diet, bins=8, color=colors[i], edgecolor='black')
ax5.set_title('Weight Loss Distribution by Diet', fontweight='bold')
ax5.set_xlabel('Weight Loss (lbs)')
ax5.set_ylabel('Frequency')
ax5.legend()
ax5.grid(axis='y', alpha=0.3)

# 6. Density Plot
ax6 = fig.add_subplot(gs[1, 2])
for i, diet in enumerate(sorted(df['diet'].unique())):
    diet_data = df[df['diet'] == diet]['weight_loss']
    diet_data.plot(kind='density', ax=ax6, label=diet, color=colors[i], linewidth=2)
ax6.set_title('Density Plot: Weight Loss Distribution', fontweight='bold')
ax6.set_xlabel('Weight Loss (lbs)')
ax6.set_ylabel('Density')
ax6.legend()
ax6.grid(alpha=0.3)

# 7. Strip Plot with Mean Diamond
ax7 = fig.add_subplot(gs[2, 0])
sns.stripplot(data=df, x='diet', y='weight_loss', alpha=0.5, size=6, ax=ax7, palette=colors)
means = df.groupby('diet')['weight_loss'].mean()
ax7.scatter(range(len(means)), means, color='red', s=400, marker='D', 
           label='Mean', zorder=10, edgecolors='darkred', linewidths=2)
ax7.set_title('Individual Values with Mean Markers', fontweight='bold')
ax7.set_xlabel('Diet Type')
ax7.set_ylabel('Weight Loss (lbs)')
ax7.legend()
ax7.grid(axis='y', alpha=0.3)

# 8. Variance Comparison
ax8 = fig.add_subplot(gs[2, 1])
variances = df.groupby('diet')['weight_loss'].var()
bars = ax8.bar(range(len(variances)), variances.values, alpha=0.8, color=colors, edgecolor='black', linewidth=1.5)
ax8.set_xticks(range(len(variances)))
ax8.set_xticklabels(variances.index)
ax8.set_title('Variance Comparison Across Diets', fontweight='bold')
ax8.set_xlabel('Diet Type')
ax8.set_ylabel('Variance')
ax8.grid(axis='y', alpha=0.3)

# Add values on bars
for i, v in enumerate(variances.values):
    ax8.text(i, v + 0.2, f'{v:.2f}', ha='center', va='bottom', fontweight='bold')

# 9. Summary Statistics Table
ax9 = fig.add_subplot(gs[2, 2])
ax9.axis('tight')
ax9.axis('off')

table_data = []
for diet in sorted(df['diet'].unique()):
    diet_data = df[df['diet'] == diet]['weight_loss']
    table_data.append([
        diet,
        f"{len(diet_data)}",
        f"{diet_data.mean():.2f}",
        f"{diet_data.std():.2f}",
        f"{diet_data.min():.2f}",
        f"{diet_data.max():.2f}"
    ])

table = ax9.table(cellText=table_data,
                 colLabels=['Diet', 'N', 'Mean', 'SD', 'Min', 'Max'],
                 cellLoc='center',
                 loc='center',
                 colColours=['lightgray']*6)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)
ax9.set_title('Summary Statistics', fontweight='bold', pad=20)

plt.savefig('diet_eda_comprehensive.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Comprehensive visualization saved as 'diet_eda_comprehensive.png'")

## 4. Check ANOVA Assumptions

### 4.1 Normality Test

In [ ]:
print("="*60)
print("NORMALITY TEST (Shapiro-Wilk Test)")
print("="*60)
print("H₀: Data is normally distributed")
print("If p > 0.05: Data appears normally distributed\n")

normality_results = []

for diet in sorted(df['diet'].unique()):
    diet_data = df[df['diet'] == diet]['weight_loss']
    stat, p_value = shapiro(diet_data)
    
    is_normal = p_value > 0.05
    
    normality_results.append({
        'Diet': diet,
        'n': len(diet_data),
        'Statistic': round(stat, 4),
        'P-value': round(p_value, 4),
        'Normal?': '✓ Yes' if is_normal else '✗ No'
    })
    
    print(f"{diet}:")
    print(f"  Sample size: n = {len(diet_data)}")
    print(f"  W-statistic: {stat:.4f}")
    print(f"  P-value: {p_value:.4f}")
    print(f"  Result: {'✓ Normal' if is_normal else '✗ Non-normal (may need non-parametric test)'}\n")

normality_df = pd.DataFrame(normality_results)
print("\nSummary:")
print(normality_df.to_string(index=False))

all_normal = all([r['Normal?'] == '✓ Yes' for r in normality_results])
print("\n" + "="*60)
if all_normal:
    print("✓ All groups appear normally distributed")
    print("  Parametric ANOVA is appropriate")
else:
    print("⚠ Some groups may not be normally distributed")
    print("  Consider: Kruskal-Wallis test (non-parametric alternative)")
    print("  Note: ANOVA is robust to moderate violations with n>20 per group")

### 4.2 Homogeneity of Variance (Levene's Test)

In [ ]:
print("="*60)
print("HOMOGENEITY OF VARIANCE (Levene's Test)")
print("="*60)
print("H₀: All groups have equal variances")
print("If p > 0.05: Variances are homogeneous\n")

groups = [df[df['diet'] == diet]['weight_loss'] for diet in sorted(df['diet'].unique())]
stat, p_value = levene(*groups)

print(f"Levene's Statistic: {stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Show individual variances
print("\nGroup Variances:")
for diet in sorted(df['diet'].unique()):
    var = df[df['diet'] == diet]['weight_loss'].var()
    std = df[df['diet'] == diet]['weight_loss'].std()
    print(f"  {diet}: σ² = {var:.3f}, σ = {std:.3f}")

# Calculate variance ratio (max/min)
variances = [df[df['diet'] == diet]['weight_loss'].var() for diet in sorted(df['diet'].unique())]
var_ratio = max(variances) / min(variances)
print(f"\nVariance Ratio (max/min): {var_ratio:.3f}")

print("\n" + "="*60)
if p_value > 0.05:
    print("✓ Variances are homogeneous across groups")
    print("  Standard ANOVA is appropriate")
else:
    print("⚠ Variances are NOT equal across groups")
    print("  Consider: Welch's ANOVA (doesn't assume equal variances)")
    
if var_ratio <= 3:
    print(f"  Variance ratio ({var_ratio:.2f}) is acceptable for ANOVA")
else:
    print(f"  Variance ratio ({var_ratio:.2f}) suggests substantial heterogeneity")

### 4.3 Visual Diagnostics

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Q-Q plots for each diet
for idx, diet in enumerate(sorted(df['diet'].unique())):
    diet_data = df[df['diet'] == diet]['weight_loss']
    stats.probplot(diet_data, dist="norm", plot=axes[0, idx])
    axes[0, idx].set_title(f'Q-Q Plot: {diet}')
    axes[0, idx].grid(True, alpha=0.3)

# Histograms with normal curve overlay
for idx, diet in enumerate(sorted(df['diet'].unique())):
    diet_data = df[df['diet'] == diet]['weight_loss']
    
    axes[1, idx].hist(diet_data, bins=10, density=True, alpha=0.7, 
                      color=colors[idx], edgecolor='black')
    
    # Overlay normal distribution
    mu, sigma = diet_data.mean(), diet_data.std()
    x = np.linspace(diet_data.min(), diet_data.max(), 100)
    axes[1, idx].plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normal')
    
    axes[1, idx].set_title(f'Histogram: {diet}')
    axes[1, idx].set_xlabel('Weight Loss (lbs)')
    axes[1, idx].set_ylabel('Density')
    axes[1, idx].legend()
    axes[1, idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('diet_normality_diagnostics.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Normality diagnostics saved as 'diet_normality_diagnostics.png'")

## 5. One-Way ANOVA

**Hypotheses:**
- **H₀**: μ₁ = μ₂ = μ₃ (all diet means are equal)
- **H₁**: At least one diet mean is different

In [ ]:
print("="*60)
print("ONE-WAY ANOVA RESULTS")
print("="*60)

# Extract diet groups
diet_groups = [df[df['diet'] == diet]['weight_loss'].values 
               for diet in sorted(df['diet'].unique())]

# Perform ANOVA
f_stat, p_value = f_oneway(*diet_groups)

print(f"\nF-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.6f}")

# Degrees of freedom
k = len(diet_groups)  # number of groups
n = len(df)  # total sample size
df_between = k - 1
df_within = n - k

print(f"\nDegrees of Freedom:")
print(f"  Between groups (df₁): {df_between}")
print(f"  Within groups (df₂): {df_within}")
print(f"  Total: {n - 1}")

# Calculate effect size (eta-squared)
grand_mean = df['weight_loss'].mean()
ss_between = sum([len(df[df['diet'] == diet]) * 
                  (df[df['diet'] == diet]['weight_loss'].mean() - grand_mean)**2 
                  for diet in df['diet'].unique()])
ss_total = sum((df['weight_loss'] - grand_mean)**2)
eta_squared = ss_between / ss_total

# Omega-squared (less biased estimate)
ms_between = ss_between / df_between
ms_within = (ss_total - ss_between) / df_within
omega_squared = (ss_between - df_between * ms_within) / (ss_total + ms_within)

print(f"\nEffect Sizes:")
print(f"  Eta-squared (η²): {eta_squared:.4f}")
print(f"  Omega-squared (ω²): {omega_squared:.4f}")

# Interpret effect size
if eta_squared < 0.01:
    effect_interpretation = "negligible"
elif eta_squared < 0.06:
    effect_interpretation = "small"
elif eta_squared < 0.14:
    effect_interpretation = "medium"
else:
    effect_interpretation = "large"

print(f"\nEffect Size: {effect_interpretation.upper()}")
print(f"({eta_squared*100:.1f}% of variance explained by diet type)")

# Statistical decision
print("\n" + "="*60)
alpha = 0.05
if p_value < alpha:
    print(f"✓ SIGNIFICANT RESULT (p = {p_value:.6f} < {alpha})")
    print("\n" + "="*60)
    print("CONCLUSION")
    print("="*60)
    print("There IS a statistically significant difference in weight loss")
    print("effectiveness among the three diets.")
    print("\nAt least one diet produces significantly different results.")
else:
    print(f"✗ NON-SIGNIFICANT RESULT (p = {p_value:.6f} >= {alpha})")
    print("\n" + "="*60)
    print("CONCLUSION")
    print("="*60)
    print("There is NO statistically significant difference in weight loss")
    print("among the three diets.")
    print("\nObserved differences are likely due to random variation.")
print("="*60)

### Detailed ANOVA Table

In [ ]:
# Using statsmodels for detailed table
model = ols('weight_loss ~ C(diet)', data=df).fit()
anova_table = anova_lm(model, typ=2)

print("\nDetailed ANOVA Table:")
print("="*60)
print(anova_table)

print("\n" + "="*60)
print("Model Fit Statistics:")
print("="*60)
print(f"R-squared: {model.rsquared:.4f}")
print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")
print(f"F-statistic: {model.fvalue:.4f}")
print(f"Prob (F-statistic): {model.f_pvalue:.6f}")

## 6. Non-Parametric Alternative: Kruskal-Wallis Test

Performed as a robustness check, especially if normality assumptions are violated.

In [ ]:
print("="*60)
print("KRUSKAL-WALLIS TEST (Non-parametric Alternative)")
print("="*60)
print("Use when: Normality assumption is violated")
print("Tests: Whether distributions differ (not just means)\n")

# Perform Kruskal-Wallis test
h_stat, kw_p_value = kruskal(*diet_groups)

print(f"H-statistic: {h_stat:.4f}")
print(f"P-value: {kw_p_value:.6f}")

print("\n" + "="*60)
if kw_p_value < 0.05:
    print("✓ SIGNIFICANT (p < 0.05)")
    print("At least one diet produces different weight loss distribution")
else:
    print("✗ NON-SIGNIFICANT (p >= 0.05)")
    print("No evidence of difference among diet distributions")

# Compare with parametric ANOVA
print("\n" + "="*60)
print("Comparison: Parametric vs Non-parametric")
print("="*60)
print(f"ANOVA F-test p-value: {p_value:.6f}")
print(f"Kruskal-Wallis p-value: {kw_p_value:.6f}")

if (p_value < 0.05) == (kw_p_value < 0.05):
    print("\n✓ Both tests agree on significance")
    print("  Results are robust across test assumptions")
else:
    print("\n⚠ Tests disagree on significance")
    print("  Consider which assumptions are better met")

## 7. Post-Hoc Analysis: Tukey HSD

In [ ]:
if p_value < 0.05:
    print("="*60)
    print("POST-HOC TEST: Tukey HSD")
    print("="*60)
    print("Purpose: Identify which specific diets differ\n")
    
    # Perform Tukey HSD
    tukey_result = pairwise_tukeyhsd(endog=df['weight_loss'], 
                                     groups=df['diet'], 
                                     alpha=0.05)
    
    print(tukey_result)
    
    # Create detailed summary
    tukey_df = pd.DataFrame(data=tukey_result.summary().data[1:], 
                           columns=tukey_result.summary().data[0])
    
    print("\n" + "="*60)
    print("Pairwise Comparison Details:")
    print("="*60)
    
    for idx, row in tukey_df.iterrows():
        g1, g2 = row['group1'], row['group2']
        meandiff = float(row['meandiff'])
        p_adj = float(row['p-adj'])
        lower = float(row['lower'])
        upper = float(row['upper'])
        reject = row['reject']
        
        mean1 = df[df['diet'] == g1]['weight_loss'].mean()
        mean2 = df[df['diet'] == g2]['weight_loss'].mean()
        
        print(f"\n{g1} vs {g2}:")
        print(f"  {g1} mean: {mean1:.2f} lbs")
        print(f"  {g2} mean: {mean2:.2f} lbs")
        print(f"  Mean difference: {meandiff:.2f} lbs")
        print(f"  95% CI: [{lower:.2f}, {upper:.2f}]")
        print(f"  Adjusted p-value: {p_adj:.4f}")
        
        if reject:
            print(f"  ✓ SIGNIFICANTLY DIFFERENT")
            better = g1 if mean1 > mean2 else g2
            print(f"  → {better} produces more weight loss")
        else:
            print(f"  ✗ Not significantly different")
else:
    print("="*60)
    print("POST-HOC TEST: Not Applicable")
    print("="*60)
    print("Overall ANOVA was not significant.")
    print("Post-hoc pairwise comparisons are not meaningful.")

### Visualize Post-Hoc Results

In [ ]:
if p_value < 0.05:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 1. Mean comparison with significance bars
    means = df.groupby('diet')['weight_loss'].mean().sort_index()
    sems = df.groupby('diet')['weight_loss'].sem().sort_index()
    
    x_pos = range(len(means))
    bars = axes[0].bar(x_pos, means, yerr=sems, capsize=10, 
                       alpha=0.8, color=colors, edgecolor='black', linewidth=2)
    axes[0].set_xticks(x_pos)
    axes[0].set_xticklabels(means.index)
    axes[0].set_title('Mean Weight Loss with Significance Bars', 
                      fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Diet Type')
    axes[0].set_ylabel('Mean Weight Loss (lbs)')
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add significance annotations
    y_max = (means + sems).max()
    annotation_height = y_max * 0.05
    
    sig_pairs = []
    for idx, row in tukey_df.iterrows():
        if row['reject']:
            g1_idx = list(means.index).index(row['group1'])
            g2_idx = list(means.index).index(row['group2'])
            sig_pairs.append((g1_idx, g2_idx, idx))
    
    for g1_idx, g2_idx, level in sig_pairs:
        y_pos = y_max + annotation_height * (level + 1) * 1.5
        axes[0].plot([g1_idx, g2_idx], [y_pos, y_pos], 'k-', linewidth=2)
        axes[0].plot([g1_idx, g1_idx], [y_pos - annotation_height*0.3, y_pos], 'k-', linewidth=2)
        axes[0].plot([g2_idx, g2_idx], [y_pos - annotation_height*0.3, y_pos], 'k-', linewidth=2)
        axes[0].text((g1_idx + g2_idx) / 2, y_pos + annotation_height*0.3, 
                    '***' if float(tukey_df.iloc[level]['p-adj']) < 0.001 else 
                    '**' if float(tukey_df.iloc[level]['p-adj']) < 0.01 else '*',
                    ha='center', va='bottom', fontsize=14, fontweight='bold')
    
    # 2. Tukey HSD confidence intervals
    tukey_result.plot_simultaneous(xlabel='Weight Loss Difference (lbs)', 
                                   ylabel='Diet Comparison', ax=axes[1])
    axes[1].set_title('Tukey HSD Simultaneous 95% Confidence Intervals',
                      fontsize=12, fontweight='bold')
    axes[1].axvline(0, color='red', linestyle='--', linewidth=2, alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('diet_posthoc_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Post-hoc visualization saved as 'diet_posthoc_analysis.png'")
else:
    print("\nPost-hoc visualization skipped (overall ANOVA not significant)")

## 8. Practical Significance Analysis

In [ ]:
print("="*60)
print("PRACTICAL SIGNIFICANCE ANALYSIS")
print("="*60)

# Calculate Cohen's d for all pairwise comparisons
def cohens_d(group1, group2):
    """Calculate Cohen's d effect size"""
    n1, n2 = len(group1), len(group2)
    var1, var2 = group1.var(), group2.var()
    pooled_std = np.sqrt(((n1-1)*var1 + (n2-1)*var2) / (n1+n2-2))
    return (group1.mean() - group2.mean()) / pooled_std

print("\nEffect Sizes (Cohen's d) for Pairwise Comparisons:")
print("(Small: 0.2, Medium: 0.5, Large: 0.8)\n")

diets = sorted(df['diet'].unique())
for i in range(len(diets)):
    for j in range(i+1, len(diets)):
        diet1_data = df[df['diet'] == diets[i]]['weight_loss']
        diet2_data = df[df['diet'] == diets[j]]['weight_loss']
        
        d = cohens_d(diet1_data, diet2_data)
        abs_d = abs(d)
        
        if abs_d < 0.2:
            magnitude = "negligible"
        elif abs_d < 0.5:
            magnitude = "small"
        elif abs_d < 0.8:
            magnitude = "medium"
        else:
            magnitude = "large"
        
        mean_diff = diet1_data.mean() - diet2_data.mean()
        
        print(f"{diets[i]} vs {diets[j]}:")
        print(f"  Mean difference: {mean_diff:.2f} lbs")
        print(f"  Cohen's d: {d:.3f}")
        print(f"  Effect size: {magnitude.upper()}\n")

# Clinical/Practical significance threshold
print("="*60)
print("Clinical Significance Interpretation:")
print("="*60)
print("Assuming >5 lbs difference is clinically meaningful...\n")

for i in range(len(diets)):
    for j in range(i+1, len(diets)):
        mean1 = df[df['diet'] == diets[i]]['weight_loss'].mean()
        mean2 = df[df['diet'] == diets[j]]['weight_loss'].mean()
        diff = abs(mean1 - mean2)
        
        if diff >= 5:
            print(f"✓ {diets[i]} vs {diets[j]}: {diff:.1f} lbs difference")
            print(f"  → Clinically meaningful difference")
        else:
            print(f"✗ {diets[i]} vs {diets[j]}: {diff:.1f} lbs difference")
            print(f"  → May not be clinically meaningful")
        print()

## 9. Final Summary and Recommendations

In [ ]:
print("="*60)
print("FINAL SUMMARY: Diet Effectiveness ANOVA Analysis")
print("="*60)

print("\n1. RESEARCH QUESTION:")
print("   Is one diet statistically more effective than others?")

print("\n2. STUDY DESIGN:")
print(f"   Total participants: n = {len(df)}")
for diet in sorted(df['diet'].unique()):
    n = len(df[df['diet'] == diet])
    mean = df[df['diet'] == diet]['weight_loss'].mean()
    sd = df[df['diet'] == diet]['weight_loss'].std()
    print(f"   {diet}: n = {n}, mean = {mean:.2f} lbs, SD = {sd:.2f}")

print("\n3. ASSUMPTION CHECKS:")
all_normal = all([r['Normal?'] == '✓ Yes' for r in normality_results])
homogeneous = p_value > 0.05
print(f"   Normality: {'✓ PASSED' if all_normal else '⚠ QUESTIONABLE'}")
print(f"   Equal variances: {'✓ PASSED' if homogeneous else '⚠ QUESTIONABLE'}")
print(f"   Independence: ✓ ASSUMED (between-subjects design)")

print("\n4. PRIMARY ANALYSIS (One-Way ANOVA):")
print(f"   F({df_between}, {df_within}) = {f_stat:.3f}")
print(f"   P-value = {p_value:.6f}")
print(f"   Effect size (η²) = {eta_squared:.4f} ({effect_interpretation})")

print("\n5. ROBUSTNESS CHECK (Kruskal-Wallis):")
print(f"   H-statistic = {h_stat:.3f}")
print(f"   P-value = {kw_p_value:.6f}")
agreement = "agree" if (p_value < 0.05) == (kw_p_value < 0.05) else "DISAGREE"
print(f"   Parametric and non-parametric tests {agreement}")

print("\n6. STATISTICAL CONCLUSION:")
if p_value < 0.05:
    print("   ✓ SIGNIFICANT DIFFERENCES DETECTED")
    print("   The weight loss varies significantly across diet types.")
    print(f"   Diet choice explains {eta_squared*100:.1f}% of weight loss variation.")
else:
    print("   ✗ NO SIGNIFICANT DIFFERENCES DETECTED")
    print("   Weight loss is similar across all three diets.")
    print("   Observed differences likely due to random chance.")

if p_value < 0.05:
    print("\n7. POST-HOC FINDINGS:")
    for idx, row in tukey_df.iterrows():
        if row['reject']:
            g1, g2 = row['group1'], row['group2']
            mean1 = df[df['diet'] == g1]['weight_loss'].mean()
            mean2 = df[df['diet'] == g2]['weight_loss'].mean()
            better = g1 if mean1 > mean2 else g2
            diff = abs(mean1 - mean2)
            print(f"   • {g1} vs {g2}: {diff:.1f} lbs difference (p = {float(row['p-adj']):.4f})")
            print(f"     → {better} is more effective")

print("\n8. PRACTICAL RECOMMENDATIONS:")
if p_value < 0.05:
    best_diet = df.groupby('diet')['weight_loss'].mean().idxmax()
    best_mean = df.groupby('diet')['weight_loss'].mean().max()
    print(f"   • {best_diet} produced the highest average weight loss ({best_mean:.1f} lbs)")
    print(f"   • Consider individual factors: adherence, preferences, health status")
    print(f"   • Statistical significance ≠ clinical significance")
    print(f"   • All diets showed positive results (weight loss occurred)")
else:
    print("   • All three diets appear equally effective")
    print("   • Choose based on personal preference and sustainability")
    print("   • Focus on adherence rather than diet type")
    print("   • Consult healthcare provider for personalized advice")

print("\n" + "="*60)
print("Analysis Complete!")
print("="*60)

## Conclusion

This comprehensive analysis evaluated whether different diet types lead to significantly different weight loss outcomes. The notebook included:

1. ✓ Thorough data exploration and visualization
2. ✓ Complete assumption checking (normality, homogeneity)
3. ✓ Primary analysis (One-Way ANOVA)
4. ✓ Robustness check (Kruskal-Wallis test)
5. ✓ Effect size calculations (η², ω², Cohen's d)
6. ✓ Post-hoc pairwise comparisons (Tukey HSD)
7. ✓ Practical significance assessment
8. ✓ Clinical recommendations

**Key Takeaway**: The analysis determines whether diet choice matters for weight loss outcomes, with specific identification of which diets (if any) are more effective than others.